In [1]:
import numpy as np
import sys
sys.path.append('..')
from utils.util import *
from utils.all_plots import *

In [ ]:
args_dict = {}
args_dict['analyze_names'] = ['calc_ratio', 'analyze_accs']

args_dict = set_args_dict(args_dict)
# all_res = read_results(args_dict)



## Read ratio results

In [ ]:
analyze_name = args_dict['analyze_names'][0]

# ----------------------
ctx_order = 'first'
ctx_order_str = 'ctxF'
# ----------------------
with open('../../results/%s_bef_c_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_bef_c = pickle.load(f)
# ----------------------
with open('../../results/%s_bef_inc_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_bef_inc = pickle.load(f)
# ----------------------
with open('../../results/%s_aft_c_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_aft_c = pickle.load(f)
# ----------------------
with open('../../results/%s_aft_inc_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_aft_inc = pickle.load(f)


    
rnn_r_runs_ctxF_bef_c    = dict_to_list(rnn_results_ctxF_bef_c, analyze_name)
rnn_r_runs_ctxF_bef_inc  = dict_to_list(rnn_results_ctxF_bef_inc, analyze_name)
rnn_r_runs_ctxF_aft_c    = dict_to_list(rnn_results_ctxF_aft_c, analyze_name)
rnn_r_runs_ctxF_aft_inc = dict_to_list(rnn_results_ctxF_aft_inc, analyze_name)

rnn_r_runs_ctxF_bef_c.keys()

In [ ]:
val_str = 'ratio_hidd'
ratio_hidds_mlp       = np.asarray(rnn_r_runs_ctxF_bef_c[val_str])

ratio_hidds_rnn_ctxF_bef_c   = np.asarray(rnn_r_runs_ctxF_bef_c[val_str])
ratio_hidds_rnn_ctxF_bef_inc = np.asarray(rnn_r_runs_ctxF_bef_inc[val_str])
ratio_hidds_rnn_ctxF_aft_c   = np.asarray(rnn_r_runs_ctxF_aft_c[val_str])
ratio_hidds_rnn_ctxF_aft_inc = np.asarray(rnn_r_runs_ctxF_aft_inc[val_str])


## Read acc results

In [ ]:
analyze_name = args_dict['analyze_names'][1]

# ----------------------
ctx_order = 'first'
ctx_order_str = 'ctxF'
# ----------------------
with open('../../results/%s_bef_c_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_bef_c = pickle.load(f)
# ----------------------
with open('../../results/%s_bef_inc_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_bef_inc = pickle.load(f)
# ----------------------
with open('../../results/%s_aft_c_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_aft_c = pickle.load(f)
# ----------------------
with open('../../results/%s_aft_inc_%s_results_rnn_sbs.P' %(analyze_name, ctx_order_str), 'rb') as f:
    rnn_results_ctxF_aft_inc = pickle.load(f)


    
rnn_acc_runs_ctxF_bef_c    = dict_to_list(rnn_results_ctxF_bef_c, analyze_name)
rnn_acc_runs_ctxF_bef_inc  = dict_to_list(rnn_results_ctxF_bef_inc, analyze_name)
rnn_acc_runs_ctxF_aft_c    = dict_to_list(rnn_results_ctxF_aft_c, analyze_name)
rnn_acc_runs_ctxF_aft_inc = dict_to_list(rnn_results_ctxF_aft_inc, analyze_name)

rnn_acc_runs_ctxF_bef_c.keys()



In [ ]:
figx, figy = 12, 6
args_dict['mi'], args_dict['mx'] = -0.02, 0.025
args_dict['sub_title'], args_dict['threshold'] = '', 1
args_dict['savefig_str'], args_dict['analyze_name'] = 'sbs_ratio_results', ''
args_dict['is_box_plot'], args_dict['is_accs_plot'] = False, False

# RNN

## before taking a step (optimization.step())

In [ ]:
acc_res_bef_c = rnn_acc_runs_ctxF_bef_c 
_, _, _, _, a_tr_bef_c, _ = get_accs(acc_res_bef_c)

acc_res_aft_c = rnn_acc_runs_ctxF_aft_c
_, _, _, _, a_tr_aft_c, _ = get_accs(acc_res_aft_c)

acc_res_bef_inc = rnn_acc_runs_ctxF_bef_inc
_, _, _, _, a_tr_bef_inc, _ = get_accs(acc_res_bef_inc)

acc_res_aft_inc = rnn_acc_runs_ctxF_aft_inc
_, _, _, _, a_tr_aft_inc, _ = get_accs(acc_res_aft_inc)


# r_tr = a_c_tr / a_inc_tr 
# df_r_a_tr = res_to_df(args_dict, r_tr)
# dfs = df_r, df_r_a_tr

In [ ]:
args_dict['val_name']='Ratio(cong/incong)'
r_res = ratio_hidds_rnn_ctxF_aft_c - ratio_hidds_rnn_ctxF_bef_c # all_res['ratio_hidds_rnn_ctxF']

acc_res = rnn_acc_runs_ctxF_bef_c # acc_res = all_res['rnn_acc_runs_ctxF']

args_dict['Label']='Distances - cong. sample'
df_r = res_to_df(args_dict, r_res)

# args_dict['Label']='Accuracy'
# a_c_tr, a_inc_tr, _, _, _, _ = get_accs(acc_res)
# r_tr = a_c_tr / a_inc_tr # ration of accs
# df_r_a_tr = res_to_df(args_dict, r_tr)
dfs = df_r#, df_r_a_tr

# plot params
args_dict['ctx_order'], args_dict['ctx_order_str'] = 'first', 'ctxF'
args_dict['model_str'], args_dict['mfig_str'] = 'RNN', 'rnn'

# plot
fig, ax = plt.subplots(figsize=(figx,figy))
boxplot_ratio_accs(dfs, args_dict, fig, ax)

# only plot the boxplot

# fig, ax = plt.subplots(figsize=(figx,figy))
# boxplot(df_r, args_dict, fig, ax)

In [ ]:
args_dict['val_name']='Ratio(cong/incong)'
r_res = ratio_hidds_rnn_ctxF_aft_inc - ratio_hidds_rnn_ctxF_bef_inc 
print(np.min(r_res), np.max(r_res))
# acc_res = rnn_acc_runs_ctxF_bef_c # acc_res = all_res['rnn_acc_runs_ctxF']

args_dict['Label']='Distances - incong. sample'
df_r = res_to_df(args_dict, r_res)

dfs = df_r

# plot params
args_dict['ctx_order'], args_dict['ctx_order_str'] = 'first', 'ctxF'
args_dict['model_str'], args_dict['mfig_str'] = 'RNN', 'rnn'

# plot
fig, ax = plt.subplots(figsize=(figx,figy))
boxplot_ratio_accs(dfs, args_dict, fig, ax)